In [1]:
import json
import requests

In [2]:

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
endpoint_url = "https://query.wikidata.org/sparql"

In [25]:
def get_results(query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_cocktails():
    # instance of IBA official cocktails
    cocktail_response = get_results("""
        SELECT 
            ?cocktail ?cocktailLabel ?cocktailDescription ?materialStmt ?quantity ?unitLabel ?ingredient ?ingredientLabel
        WHERE {   
            ?cocktail wdt:P31 wd:Q2536409;
                    p:P186 ?materialStmt.
            ?materialStmt ps:P186 ?ingredient;
                        pqv:P1114 [wikibase:quantityAmount ?quantity; wikibase:quantityUnit ?unit ].
            MINUS { ?material wdt:P31 wd:Q81727. }
            SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } ORDER BY (?cocktail)
    """)

    # return [
    #     { 'name': cocktail['cocktailLabel']['value'], 
    #         'uri:': cocktail['cocktail']['value'] } 
    #     for cocktail in cocktail_response['results']['bindings']
    # ]
    return cocktail_response

def parse_cocktails(cocktail_data):
    cocktails = [
        {
            "url": cocktail['cocktail']['value'],
            "title": cocktail['cocktailLabel']['value'],
            "description": cocktail.get('cocktailDescription', 'no description')
        }
        for cocktail in cocktail_data['results']['bindings']
    ]



In [26]:
cocktails = get_cocktails()
print(parse_cocktails(cocktails)[:5])

TypeError: 'NoneType' object is not subscriptable